In [ ]:
df = self.train_data[self.train_data['fips'] == 1001]


df['year'] = pd.to_datetime(df['date']).dt.year
df['week_num'] = pd.to_datetime(df['date']).dt.week

df['score_day'] = df['score'].apply(lambda x: 'yes' if pd.notnull(x) == True else '')

aggregated_data_train = df.groupby(['fips','year','week_num']).agg(
                                {'PRECTOT': ['mean'],
                                'PS': ['mean'],
                                'QV2M': ['mean'],
                                'T2M': ['mean'],
                                'T2MDEW': ['mean'],
                                'T2MWET': ['mean'],
                                'T2M_MAX': ['mean'],
                                'T2M_MIN': ['mean'],
                                'T2M_RANGE': ['mean'],
                                 'TS': ['mean'],
                                 'WS10M': ['mean'],
                                 'WS10M_MAX': ['mean'],
                                 'WS10M_MIN': ['mean'],
                                 'WS10M_RANGE': ['mean'],
                                 'WS50M': ['mean'],
                                 'WS50M_MAX': ['mean'],
                                 'WS50M_MIN': ['mean'],
                                 'WS50M_RANGE': ['mean'],
                                 'score': 'max'}).reset_index().sort_values(['fips','year','week_num'])

aggregated_data_train.columns = ['_'.join(col) for col in aggregated_data_train.columns.values]
aggregated_data_train['score_max'] = aggregated_data_train['score_max'].map(lambda x: np.round(x))

fips_dict = self.fips_dict.drop(columns=['COUNTYNAME',"STATE",'geom']).rename(columns={'fips':'fips_'})
fips_dict["lat_long"] = fips_dict["lat_long"].transform(lambda x: ast.literal_eval(x))
fips_dict["lat"] = pd.DataFrame(fips_dict["lat_long"].tolist())[0]
fips_dict["long"] = pd.DataFrame(fips_dict["lat_long"].tolist())[1]
fips_dict.drop(columns=["lat_long"],inplace=True)
aggregated_data_train = pd.merge(aggregated_data_train,fips_dict, on=["fips_"], how="inner")
aggregated_data_train = aggregated_data_train[['fips_', 'year_', 'week_num_', 'PRECTOT_mean', 'PS_mean', 'QV2M_mean',
'T2M_mean', 'T2MDEW_mean', 'T2MWET_mean', 'T2M_MAX_mean',
'T2M_MIN_mean', 'T2M_RANGE_mean', 'TS_mean', 'WS10M_mean',
'WS10M_MAX_mean', 'WS10M_MIN_mean', 'WS10M_RANGE_mean', 'WS50M_mean',
'WS50M_MAX_mean', 'WS50M_MIN_mean', 'WS50M_RANGE_mean','lat', 'long','score_max']]


In [5]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.tree import DecisionTreeClassifier
import ast


file_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
full_path_train = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'train_timeseries.csv')
full_path_validate = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'validation_timeseries.csv')
full_path_test = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'test_timeseries.csv')
full_path_fips = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'fips_dict.csv')

train_data = pd.read_csv(full_path_train)[2:]
df = train_data[train_data['fips'] == 1001]

In [6]:
df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN
5,1001,2000-01-06,0.01,101.31,3.49,4.99,-1.11,-1.07,12.89,-2.96,...,3.32,1.55,2.39,0.04,2.35,2.95,5.22,0.05,5.17,NaN
6,1001,2000-01-07,0.01,101.37,3.93,5.99,0.55,0.58,14.51,0.63,...,5.69,2.31,3.28,1.59,1.69,5.02,6.47,2.44,4.03,NaN


In [7]:
df['year'] = pd.to_datetime(df['date']).dt.year
df['week_num'] = pd.to_datetime(df['date']).dt.week
df.head()

/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/202400404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.to_datetime(df['date']).dt.year
/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/202400404.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_num'] = pd.to_datetime(df['date']).dt.week
/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/202400404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score,year,week_num
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN,2000,1
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0,2000,1
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN,2000,1
5,1001,2000-01-06,0.01,101.31,3.49,4.99,-1.11,-1.07,12.89,-2.96,...,2.39,0.04,2.35,2.95,5.22,0.05,5.17,NaN,2000,1
6,1001,2000-01-07,0.01,101.37,3.93,5.99,0.55,0.58,14.51,0.63,...,3.28,1.59,1.69,5.02,6.47,2.44,4.03,NaN,2000,1


In [18]:
weeks_in_year = 52

df['sin_week'] = np.sin(2*np.pi*df.week_num/weeks_in_year)
df['cos_week'] = np.cos(2*np.pi*df.week_num/weeks_in_year)

/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/2909890430.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sin_week'] = np.sin(2*np.pi*df.week_num/weeks_in_year)
/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/2909890430.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cos_week'] = np.cos(2*np.pi*df.week_num/weeks_in_year)


In [21]:
df[df.fips==1001].head(50)

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score,year,week_num,sin_week,cos_week
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,2.67,7.53,9.52,5.87,3.66,NaN,2000,1,0.120537,0.992709
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,3.59,6.73,9.31,3.74,5.58,1.0,2000,1,0.120537,0.992709
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,1.98,2.94,4.85,0.65,4.19,NaN,2000,1,0.120537,0.992709
5,1001,2000-01-06,0.01,101.31,3.49,4.99,-1.11,-1.07,12.89,-2.96,...,2.35,2.95,5.22,0.05,5.17,NaN,2000,1,0.120537,0.992709
6,1001,2000-01-07,0.01,101.37,3.93,5.99,0.55,0.58,14.51,0.63,...,1.69,5.02,6.47,2.44,4.03,NaN,2000,1,0.120537,0.992709
7,1001,2000-01-08,1.02,100.77,5.71,8.69,5.33,5.34,15.78,2.74,...,1.40,4.17,5.73,2.01,3.72,NaN,2000,1,0.120537,0.992709
8,1001,2000-01-09,16.19,99.88,11.18,15.73,15.45,15.45,19.60,10.30,...,1.73,5.97,7.44,4.95,2.49,NaN,2000,1,0.120537,0.992709
9,1001,2000-01-10,21.23,99.78,7.77,13.61,9.59,9.60,17.65,7.73,...,3.88,6.33,10.03,4.15,5.88,NaN,2000,2,0.239316,0.970942
10,1001,2000-01-11,1.33,100.40,6.63,11.48,7.84,7.84,18.88,5.72,...,1.43,3.55,6.38,1.71,4.67,2.0,2000,2,0.239316,0.970942
11,1001,2000-01-12,0.17,100.59,8.39,13.02,11.36,11.36,20.74,6.43,...,1.31,5.05,7.27,2.10,5.17,NaN,2000,2,0.239316,0.970942


In [25]:
BUCKET_NAME='drought-modelling-models'
DATA_BUCKET_NAME = 'drought-modelling-datasets'
BUCKET_TRAIN_DATA_PATH = 'data/train_timeseries.csv'
BUCKET_VAL_DATA_PATH = 'data/validation_timeseries.csv'
BUCKET_TEST_DATA_PATH = 'data/test_timeseries.csv'
BUCKET_FIPS_PATH = 'data/fips_dict.csv'

class DataFunctions():
    
    def __init__(self, local=False):
        
        file_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
        full_path_train = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'train_timeseries.csv')
        full_path_validate = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'validation_timeseries.csv')
        full_path_test = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'test_timeseries.csv')
        full_path_fips = os.path.join(file_path,'realGhostFoxx','droughts_modelling', 'raw_data', 'fips_dict.csv')
        
        if local:
            self.train_data = pd.read_csv(full_path_train)[2:]
            self.validation_data = pd.read_csv(full_path_validate)[1:]
            self.test_data = pd.read_csv(full_path_test)[6:]
            self.fips_dict = pd.read_csv(full_path_fips,index_col=[0])
        else:
            self.train_data = pd.read_csv(f"gs://{DATA_BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH}")
            self.validation_data = pd.read_csv(f"gs://{DATA_BUCKET_NAME}/{BUCKET_VAL_DATA_PATH}")
            self.test_data = pd.read_csv(f"gs://{DATA_BUCKET_NAME}/{BUCKET_TEST_DATA_PATH}")
            self.fips_dict = pd.read_csv(f"gs://{DATA_BUCKET_NAME}/{BUCKET_FIPS_PATH}")
          
    def light_weekly_aggregate_train(self, scope='all'):
        
        if scope != 'all':
            df = self.train_data[self.train_data['fips'] == 1001]
        else:
            df = self.train_data
   
        df['year'] = pd.to_datetime(df['date']).dt.year
        df['week_num'] = pd.to_datetime(df['date']).dt.week

        df['score_day'] = df['score'].apply(lambda x: 'yes' if pd.notnull(x) == True else '')

        aggregated_data_train = df.groupby(['fips','year','week_num']).agg(
                                        {'PRECTOT': ['mean'],
                                        'PS': ['mean'],
                                        'QV2M': ['mean'],
                                        'T2M': ['mean'],
                                        'T2MDEW': ['mean'],
                                        'T2MWET': ['mean'],
                                        'T2M_MAX': ['mean'],
                                        'T2M_MIN': ['mean'],
                                        'T2M_RANGE': ['mean'],
                                         'TS': ['mean'],
                                         'WS10M': ['mean'],
                                         'WS10M_MAX': ['mean'],
                                         'WS10M_MIN': ['mean'],
                                         'WS10M_RANGE': ['mean'],
                                         'WS50M': ['mean'],
                                         'WS50M_MAX': ['mean'],
                                         'WS50M_MIN': ['mean'],
                                         'WS50M_RANGE': ['mean'],
                                         'score': 'max'}).reset_index().sort_values(['fips','year','week_num'])

        aggregated_data_train.columns = ['_'.join(col) for col in aggregated_data_train.columns.values]
        aggregated_data_train['score_max'] = aggregated_data_train['score_max'].map(lambda x: np.round(x))
        
        fips_dict = self.fips_dict.drop(columns=['COUNTYNAME',"STATE",'geom']).rename(columns={'fips':'fips_'})
        fips_dict["lat_long"] = fips_dict["lat_long"].transform(lambda x: ast.literal_eval(x))
        fips_dict["lat"] = pd.DataFrame(fips_dict["lat_long"].tolist())[0]
        fips_dict["long"] = pd.DataFrame(fips_dict["lat_long"].tolist())[1]
        fips_dict.drop(columns=["lat_long"],inplace=True)
        aggregated_data_train = pd.merge(aggregated_data_train,fips_dict, on=["fips_"], how="inner")
        
        aggregated_data_train['sin_week_num'] = np.sin(2*np.pi*aggregated_data_train.week_num/52)
        aggregated_data_train['cos_week_num'] = np.cos(2*np.pi*aggregated_data_train.week_num/52)
        
        aggregated_data_train = aggregated_data_train[['fips_', 'year_', 'sin_week_num', 'cos_week_num', 'PRECTOT_mean', 'PS_mean', 'QV2M_mean',
       'T2M_mean', 'T2MDEW_mean', 'T2MWET_mean', 'T2M_MAX_mean',
       'T2M_MIN_mean', 'T2M_RANGE_mean', 'TS_mean', 'WS10M_mean',
       'WS10M_MAX_mean', 'WS10M_MIN_mean', 'WS10M_RANGE_mean', 'WS50M_mean',
       'WS50M_MAX_mean', 'WS50M_MIN_mean', 'WS50M_RANGE_mean','lat', 'long','score_max']]
        
        return aggregated_data_train.dropna()

In [27]:
cat = DataFunctions(local=True)

_request out of retries on exception: Cannot connect to host storage.googleapis.com:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1124)')]
Traceback (most recent call last):
  File "/Users/voxxy/.pyenv/versions/droughts/lib/python3.8/site-packages/aiohttp/connector.py", line 969, in _wrap_create_connection
    return await self._loop.create_connection(*args, **kwargs)  # type: ignore  # noqa
  File "/Users/voxxy/.pyenv/versions/3.8.6/lib/python3.8/asyncio/base_events.py", line 1050, in create_connection
    transport, protocol = await self._create_connection_transport(
  File "/Users/voxxy/.pyenv/versions/3.8.6/lib/python3.8/asyncio/base_events.py", line 1080, in _create_connection_transport
    await waiter
  File "/Users/voxxy/.pyenv/versions/3.8.6/lib/python3.8/asyncio/sslproto.py", line 529, in data_received
    ssldata, appdata = self._sslpipe.feed_ssldata(data)
  File "/Users

In [ ]:
df2 = cat.light_weekly_aggregate_train()

/var/folders/kh/fd0llhc91_s537ztm8vty5q00000gn/T/ipykernel_24500/1295774877.py:37: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_num'] = pd.to_datetime(df['date']).dt.week


In [ ]:
df2.head()